In [97]:
import json
import os
import requests
import io
import shutil
import math
import PIL
import pprint
import numpy as np
import glob
import shutil
import PIL.Image as PImage
from PIL import ImageEnhance
from pprint import pprint
from collections import Counter
from datetime import datetime
from PIL import Image, ImageOps, ImageEnhance
from sklearn.model_selection import train_test_split

In [98]:
API_BASE_URL = 'http://fireeye-test-backend-container:9090/api/'
TF_SERVING_BASE_URL = 'http://fireeye-test-model-container:8501/'
task_id='1ac1e8a095df4611af387d9934799251'
id_code_mapping = {
    'dbee3deebc5444f5b011da4e5518752c': '0',
    'edb4cb51d54644c08aa122d3f041bb0a': '1'}

In [99]:
num_images = requests.get(
    url=API_BASE_URL+'image/count', 
    params=dict(
        task_id=task_id,
        has_truth=True
    )
).json()

print('该图片数量：',num_images)

该图片数量： 318


In [100]:
import pprint

def get_image_records(task_id):
    resp = requests.get(
        url=API_BASE_URL+'image', 
        params=dict(
            task_id=task_id,
            has_truth=True
        )
    )
    if resp.status_code == 200:
        return resp.json()
    else:
        raise RuntimeError(resp.text)

image_records=get_image_records(task_id)

In [101]:
image_dir = "./images"
category0_dir = os.path.join(image_dir, 'Category0')
category1_dir = os.path.join(image_dir, 'Category1')
if not os.path.exists(category0_dir):
    os.makedirs(category0_dir)

if not os.path.exists(category1_dir):
    os.makedirs(category1_dir)

In [102]:
def color_jitter(img: Image.Image, brightness=0.2, contrast=0.2, saturation=0.2) -> Image.Image:
    img = ImageEnhance.Brightness(img).enhance(1 + brightness * (2 * np.random.random() - 1))
    img = ImageEnhance.Contrast(img).enhance(1 + contrast * (2 * np.random.random() - 1))
    img = ImageEnhance.Color(img).enhance(1 + saturation * (2 * np.random.random() - 1))
    return img

In [103]:
def resize_image(img: Image.Image, target_size=(224, 224)) -> Image.Image:
    return img.resize(target_size, Image.ANTIALIAS)

In [104]:
def download_image(image_id):
    response = requests.get(f"{API_BASE_URL}image/download/{image_id}")
    return response.content

In [105]:
def vertical_flip(img: Image.Image) -> Image.Image:
    return ImageOps.flip(img)

In [106]:
def horizontal_flip(img: Image.Image) -> Image.Image:
    return ImageOps.mirror(img)

In [107]:
def normalize_image(img: Image.Image) -> np.ndarray:
    img_array = np.array(img)
    return img_array / 255.0

In [108]:
def get_image_by_id(id):
    r = requests.get(url=API_BASE_URL+'image/'+id)
    if r.status_code == 200:
        return PIL.Image.open(io.BytesIO(r.content))
    else:
        raise RuntimeError(r.text)
img = get_image_by_id(image_records[35]['id']) 
img_vertical_flipped = vertical_flip(img)
img_horizontal_flipped = horizontal_flip(img)
img_normalized = normalize_image(img)
jittered_image = color_jitter(img)
resized_image = resize_image(img)

In [109]:
#可省略步骤，主要看看效果
#上面一段还需要研究一下，是否删除
img_vertical_flipped.save("path_to_save_vertical_flipped.png")
img_horizontal_flipped.save("path_to_save_horizontal_flipped.png")
jittered_image.save('path_to_save_jittered_image.png')
resized_image.save('path_to_save_resized_image.png')

In [ ]:
for record in image_records:
    image_content = download_image(record['id'])
    truth_id = record['truth_id']

    # Note: `id_code_mapping` is not defined in the provided code.
    # Make sure it's properly defined or replaced by appropriate logic.
    if id_code_mapping[truth_id] == '0':
        file_path = os.path.join(category0_dir, f"{record['id']}.png")
    else:
        file_path = os.path.join(category1_dir, f"{record['id']}.png")
    try:
        img = Image.open(io.io.BytesIO(image_content))
        img.save(file_path, 'PNG')
    except Exception as e:
        print(f'Error processing image {record["id"]}. Error: {e}')

    with open(file_path, 'wb') as f:
        f.write(image_content)
    img.save(file_path, 'PNG')

In [96]:
def preprocess_and_save_images(image_list, save_dir):
    for image_path in image_list:
        try:
            img = Image.open(image_path)
            img_vertical_flipped = vertical_flip(img)
            img_horizontal_flipped = horizontal_flip(img)
            img_normalized = normalize_image(img)
            img_resized = resize_image(img)
            img_jittered = color_jitter(img)

            base_name = os.path.basename(image_path).split('.')[0]
            img_vertical_flipped.save(os.path.join(save_dir, f"{base_name}_vertical_flipped.png"))
            img_horizontal_flipped.save(os.path.join(save_dir, f"{base_name}_horizontal_flipped.png"))
            img_resized.save(os.path.join(save_dir, f"{base_name}_resized.png"))
            img_jittered.save(os.path.join(save_dir, f"{base_name}_jittered.png"))

            img_normalized_pil = Image.fromarray((img_normalized * 255).astype('uint8'))
            img_normalized_pil.save(os.path.join(save_dir, f"{base_name}_normalized.png"))
        except Exception as e:
            print(f'Error processing image {image_path}. Error: {e}')



all_category0_images = [os.path.join(category0_dir, fname) for fname in os.listdir(category0_dir)]
all_category1_images = [os.path.join(category1_dir, fname) for fname in os.listdir(category1_dir)]

preprocess_and_save_images(all_category0_images, category0_dir)
preprocess_and_save_images(all_category1_images, category1_dir)

category0_train, category0_val, category0_test = split_data(category0_dir)
category1_train, category1_val, category1_test = split_data(category1_dir)

Error processing image ./images/Category0/.ipynb_checkpoints. Error: [Errno 21] Is a directory: './images/Category0/.ipynb_checkpoints'


In [71]:
def split_data(directory):
    all_files = [os.path.join(directory, fname) for fname in os.listdir(directory)]
    train_files, test_files = train_test_split(all_files, test_size=0.2, random_state=42)
    train_files, val_files = train_test_split(train_files, test_size=0.25, random_state=42)
    return train_files, val_files, test_files

In [79]:
category0_train, category0_val, category0_test = split_data(category0_dir)
category1_train, category1_val, category1_test = split_data(category1_dir)

def move_files(file_list, target_dir):
    if os.path.exists(target_dir):
        shutil.rmtree(target_dir)
    os.makedirs(target_dir)

    for f in file_list:
        shutil.move(f, target_dir)

move_files(category0_train, os.path.join(train_dir, 'Category0'))
move_files(category0_val, os.path.join(val_dir, 'Category0'))
move_files(category0_test, os.path.join(test_dir, 'Category0'))

move_files(category1_train, os.path.join(train_dir, 'Category1'))
move_files(category1_val, os.path.join(val_dir, 'Category1'))
move_files(category1_test, os.path.join(test_dir, 'Category1'))

NameError: name 'train_files' is not defined

In [302]:
import tensorflow as tf
from tensorflow.keras import layers

print(tf.__version__)

2.8.2


In [45]:
batch_size = 32

In [10]:
#ef build_dataset(data_dir, target=None):
#   all_images = []
#   all_labels = []
#   for i in range(4):
#       if target is not None and i != target:
#           continue
#       images = glob.glob(os.path.join(data_dir, f'{i}/*.png'), recursive=True)
#       all_images.extend(images)
#       all_labels.extend([i] * len(images))
#   ds = tf.data.Dataset.from_tensor_slices((all_images, all_labels))
#   ds = ds.map(lambda x,y: (tf.io.encode_base64(tf.io.read_file(x)) ,y))
#   ds = ds.shuffle(100, seed=123)
#   ds = ds.batch(batch_size)
#   return ds


#rain_ds = build_dataset(train_dir)
#ev_ds = build_dataset(dev_dir)
#test_ds = build_dataset(test_dir)

2023-09-04 06:48:50.861283: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-09-04 06:48:50.861318: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-09-04 06:48:50.861336: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (58c881efe57c): /proc/driver/nvidia/version does not exist
2023-09-04 06:48:50.862537: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [370]:
print(train_ds)
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

NameError: name 'train_ds' is not defined

In [12]:
#from keras import models
#

#def process_base64_image(s):
#    img = tf.io.decode_base64(s)
#    img = tf.io.decode_png(img, channels=3)
#    img = tf.image.resize(img, (img_height, img_width), antialias=True)    
#    return img

#model = models.Sequential([
 #   layers.Lambda(
 #           (
#                lambda x: tf.map_fn(
#                    process_base64_image,
#                    x,
#                    fn_output_signature=tf.TensorSpec(shape=(int(img_height), int(img_width), 3), dtype=tf.float32))
#            ),
#            name='decode_base64_png'
#        ),
#    tf.keras.layers.Conv2D(32, (3, 3),  activation='relu', input_shape=(150, 150, 3)),
#    tf.keras.layers.MaxPool2D(2, 2),
#    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
#    tf.keras.layers.MaxPooling2D(2, 2),
#    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
#    tf.keras.layers.MaxPooling2D(2, 2),
#    tf.keras.layers.Flatten(),
#    tf.keras.layers.Dense(512, activation='relu'),
#    tf.keras.layers.Dense(1, activation='sigmoid')])

#from keras import optimizers
#model.compile(
#    loss='binary_crossentropy',
#    optimizer=optimizers.RMSprop(lr=1e-4),
#    metrics=['accuracy'])
#model.summary()
#model.compile(
#  optimizer='adam',
#  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
#  metrics=['accuracy'])


In [16]:
def process_base64_image(s):
    img = tf.io.decode_base64(s)
    img = tf.io.decode_png(img, channels=3)
    img = tf.image.resize(img, (img_height, img_width), antialias=True)    
    return img

# 构建模型
model = tf.keras.Sequential([
    layers.Lambda(
            (
                lambda x: tf.map_fn(
                    process_base64_image,
                    x,
                    fn_output_signature=tf.TensorSpec(shape=(int(img_height), int(img_width), 3), dtype=tf.float32))
            ),
            name='decode_base64_png'
        ),
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(img_height, img_width, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=(img_height, img_width, 3)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(2)
])

In [17]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [18]:
epochs = 20
history = model.fit(
    train_ds,
    epochs=epochs,
    validation_data=dev_ds
)

Epoch 1/20
15/15 [==============================] - 12s 726ms/step - loss: 295.4449 - accuracy: 0.7134 - val_loss: 51.6766 - val_accuracy: 0.3291
Epoch 2/20
15/15 [==============================] - 11s 727ms/step - loss: 28.4405 - accuracy: 0.7909 - val_loss: 53.9243 - val_accuracy: 0.6709
Epoch 3/20
15/15 [==============================] - 11s 723ms/step - loss: 26.9080 - accuracy: 0.6293 - val_loss: 3.1529 - val_accuracy: 0.3367
Epoch 4/20
15/15 [==============================] - 11s 724ms/step - loss: 2.0840 - accuracy: 0.7823 - val_loss: 1.6101 - val_accuracy: 0.6709
Epoch 5/20
15/15 [==============================] - 11s 725ms/step - loss: 0.8889 - accuracy: 0.6034 - val_loss: 8.1377 - val_accuracy: 0.6709
Epoch 6/20
15/15 [==============================] - 11s 725ms/step - loss: 2.2395 - accuracy: 0.6164 - val_loss: 0.7689 - val_accuracy: 0.6709
Epoch 7/20
15/15 [==============================] - 11s 726ms/step - loss: 0.5435 - accuracy: 0.7091 - val_loss: 0.4800 - val_accuracy: 

In [19]:
model.evaluate(test_ds)

12/12 [==============================] - 3s 220ms/step - loss: 0.3729 - accuracy: 0.7135


[0.3728915750980377, 0.7134831547737122]

In [ ]:
import pytz
from datetime import datetime

model_version =  datetime.now(pytz.timezone('Asia/Shanghai')).strftime('%Y%m%d%H%M%S')
tf.keras.models.save_model(
    model,
    f'/models/slot1/{model_version}/',
    overwrite=True,
)

In [ ]:
import base64

def predict_image(images):    
    bimages = []
    for image in images:
        with open(image, 'rb') as  fimage:
            content = fimage.read()
        bimage = base64.urlsafe_b64encode(content).decode()
        bimages.append(bimage)
    req_data ={
      'inputs': bimages,
    } 
    response = requests.post(TF_SERVING_BASE_URL+f'v1/models/slot1/versions/{model_version}:predict', # 根据部署地址填写
                             json=req_data,
                             headers={"content-type": "application/json"})
    if response.status_code != 200:
        raise RuntimeError('Request tf-serving failed: ' + response.text)
    resp_data = json.loads(response.text)    
    if 'outputs' not in resp_data \
                        or type(resp_data['outputs']) is not list:
        raise ValueError('Malformed tf-serving response')
    outputs = np.argmax(resp_data['outputs'], axis=1).tolist()
    return outputs


def test_image_model(test_dir, code, batch_size=10):    
    images = list(pathlib.Path(test_dir).joinpath(str(code)).glob('./*.png')) 
    codes = []
    for step in range(math.ceil(len(images)/batch_size)):
        outputs = predict_image(images[step*batch_size:(step+1)*batch_size])
        for i, o in zip(images, outputs):            
            if o != code:
                print('错误图片：', i)
        codes.extend(outputs)
    accuracy = round(codes.count(code) / len(codes), 4)
    return accuracy, codes

accuracy, codes = test_image_model(test_dir, 0)
print('类别0的准确率', accuracy)
print('类别0的测试结果', codes)
accuracy, codes = test_image_model(test_dir, 1)
print('类别1的准确率', accuracy)
print('类别1的测试结果', codes)